# Resistor Value Prediction CNN Model

## 0. Getting Setup

In [ ]:
# Mount our google drive to authorize access
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
# Import the need packages

import torch
import torchvision


from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it does not work
try:
  from torchinfo import summary
except:
  print("[INFO] Couldn't find torchinfo... installing it.")
  !pip install -q torchinfo
  from torchinfo import summary

# Try to import the going_modular directory from Mr. Bourke's Pytorch training
try:
  from going_modular.going_modular import data_setup, engine
  from helper_functions import download_data, set_seeds, plot_loss_curves
except:
  #Get the going modular_scripts
  print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
  !git clone https://github.com/mrdbourke/pytorch-deep-learning
  !mv pytorch-deep-learning/going_modular .
  !mv pytorch-deep-learning/helper_functions.py . # get the helper_functions.py script
  !rm -rf pytorch-deep-learning
  from going_modular.going_modular import data_setup, engine
  from helper_functions import download_data, set_seeds, plot_loss_curves

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

## 1. Get and Preprocess Data

In [ ]:
#Install kaggle package
!pip install kaggle

# Upload my kaggle.json file
from google.colab import files
uploaded = files.upload() # Click "Choose Files" and select kaggle.json

# Set up the credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Now download your specific dataset
!kaggle datasets download -d eralpozcan/resistor-dataset
!unzip resistor-dataset.zip -d /content/resistor_dataset/

In [ ]:
# Combine all image files from my directory
import os
import shutil
import glob

# Create main dataset directory
main_dir = '/content/resistor_dataset/'
os.makedirs(main_dir, exist_ok=True)


def move_files(src_dir, dst_dir):
  """Find all image files recursively and move them
  """
  for root, dirs, files in os.walk(src_dir):
    for file in files:
        if file.endswith(('.jpg', '.png', '.jpeg')):
            src = os.path.join(root, file)
            dst = os.path.join(dst_dir, file)
            shutil.move(src, dst)

# Find all image files recursively and move them
#move_files('resistor_dataset', main_dir)
print(f"Moved all images to {main_dir}")


In [ ]:
# Remove unecessary files

!rm -f resistor-dataset.zip
!rm -f *.jpg *.png *.csv *.txt
!rm -rf kaggle.json

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
import glob

"""
# Split the data into train and test sets
train_dir = '/content/resistor_dataset/train'
test_dir = '/content/resistor_dataset/test'

# Create the directory structure
try:
  os.makedirs(train_dir)
  os.makedirs(test_dir)
except FileExistsError:
  print("Directory already exists")


# Get all image files from all_resistors
all_images = [f for f in os.listdir



    ('/content/resistor_dataset') if f.endswith(('.jpg','.png', '.jpeg'))]

# Split the image names
try:
  train_images, test_images = train_test_split(all_images, test_size=0.2, random_state=42)
  # Move training images
  for img in train_images:
   src = os.path.join('/content/resistor_dataset', img)
   dst = os.path.join(train_dir, img)
   shutil.move(src, dst)

  # Move testing images
  for img in test_images:
    src = os.path.join('/content/resistor_dataset/*', img)
    dst = os.path.join(test_dir, img)
    shutil.move(src, dst)

  # Delete the old images
  # Remove original images after copying
  for file in glob.glob('/content/resistor_dataset/*/*.jpg'):
    os.remove(file)
  for file in glob.glob('/content/resistor_dataset/*/*.png'):
    os.remove(file)
except:
    print("The data has already been split ")
"""


In [ ]:
import os
import shutil
import random
from pathlib import Path

def simple_train_test_split(dataset_path, train_ratio=0.8):
    """
    Simple function to split dataset into train/test
    """
    dataset_path = Path(dataset_path)

    # Create train and test directories
    train_dir = dataset_path.parent / "all_resistors" / "train"
    test_dir = dataset_path.parent / "all_resistors" / "test"

    train_dir.mkdir(parents=True, exist_ok=True)
    test_dir.mkdir(parents=True, exist_ok=True)

    # Process each class folder
    for class_folder in dataset_path.iterdir():
        if not class_folder.is_dir():
            continue

        class_name = class_folder.name
        print(f"Processing class: {class_name}")

        # Create class subdirectories
        (train_dir / class_name).mkdir(exist_ok=True)
        (test_dir / class_name).mkdir(exist_ok=True)

        # Get all images in this class
        images = list(class_folder.glob("*.jpg")) + list(class_folder.glob("*.png"))
        random.shuffle(images)

        # Split images
        split_point = int(len(images) * train_ratio)
        train_images = images[:split_point]
        test_images = images[split_point:]

        # Copy to train folder
        for img in train_images:
            shutil.copy2(img, train_dir / class_name / img.name)

        # Copy to test folder
        for img in test_images:
            shutil.copy2(img, test_dir / class_name / img.name)

        print(f"  Train: {len(train_images)}, Test: {len(test_images)}")

# Use it like this:
simple_train_test_split("/content/resistor_dataset", train_ratio=0.8)

### 1.2 Create transforms for data

In [ ]:
# We are going to be using four models and select the best performing model for deployment
  # - ResNet-18
  # - ResNet34
  # - EffNEtB0
  # - EffNetB2
  # - MobileNetV2

import torch
import torchvision
from torchvision import transforms

# Setup pretrained weights
resnet18_weights = torchvision.models.ResNet18_Weights.DEFAULT
resnet34_weights = torchvision.models.ResNet34_Weights.DEFAULT
effnetb0_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
mobilenetv2_weights = torchvision.models.MobileNet_V2_Weights.DEFAULT

# Get the transforms for each model from the weights
resnet18_transforms = resnet18_weights.transforms()
resnet34_transforms = resnet34_weights.transforms()
effnetb0_transforms = effnetb0_weights.transforms()
effnetb2_transforms = effnetb2_weights.transforms()
mobilenetv2_transforms = mobilenetv2_weights.transforms()


def train_transform(base_transform: transforms.Compose):
  """Create a training transform with data augmentation
  """
  transform = transforms.Compose([
      transforms.TrivialAugmentWide(num_magnitude_bins=31),
      transforms.RandomHorizontalFlip(p=0.5),
      transforms.RandomVerticalFlip(p=0.5),
      transforms.RandomRotation(degrees=(-30, 30)),
      transforms.ToTensor(),
      base_transform
  ])

  return transform



In [ ]:
resnet18_transforms

### 1.3 Create DataLoaders

In [ ]:
"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    train_transform: transforms.Compose,
    test_transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """
  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=train_transform)
  test_data = datasets.ImageFolder(test_dir, transform=test_transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names


In [ ]:
# Setup parameters
BATCH_SIZE=32
train_dir = '/content/all_resistors/train'
test_dir = '/content/all_resistors/test'


# Create individual dataloaders for each model
resnet18_train_dataloader, resnet18_test_dataloader, resnet18_class_names = create_dataloaders(train_dir=train_dir,
                                                                                                          test_dir=test_dir,
                                                                                                          train_transform=train_transform(resnet18_transforms),
                                                                                                          test_transform=resnet18_transforms,
                                                                                                          batch_size=BATCH_SIZE)

resnet34_train_dataloader, resnet34_test_dataloader, resnet34_class_names = create_dataloaders(train_dir=train_dir,
                                                                                                          test_dir=test_dir,
                                                                                                          train_transform=train_transform(resnet34_transforms),
                                                                                                          test_transform=resnet34_transforms,
                                                                                                          batch_size = BATCH_SIZE)

effnetb0_train_dataloader, effnetb0_test_dataloader, effnetb0_class_names = create_dataloaders(train_dir=train_dir,
                                                                                                          test_dir=test_dir,
                                                                                                          train_transform=train_transform(effnetb0_transforms),
                                                                                                          test_transform=effnetb0_transforms,
                                                                                                          batch_size = BATCH_SIZE)

effnetb2_train_dataloader, effnetb2_test_dataloader, effnetb2_class_names = create_dataloaders(train_dir=train_dir,
                                                                                                          test_dir=test_dir,
                                                                                                          train_transform=train_transform(effnetb2_transforms),
                                                                                                          test_transform=effnetb2_transforms,
                                                                                                          batch_size=BATCH_SIZE)
mobile_train_dataloader, mobile_test_dataloader, mobile_class_names = create_dataloaders(train_dir= train_dir,
                                                                                                    test_dir=test_dir,
                                                                                                    train_transform=train_transform(mobilenetv2_transforms),
                                                                                                    test_transform=mobilenetv2_transforms,
                                                                                                    batch_size=BATCH_SIZE)

In [ ]:
len(mobile_class_names)

In [ ]:
model = torchvision.models.mobilenet_v2(weights=mobilenetv2_weights).to(device)

summary(model, (32, 3, 224, 224))

### 1.4 Creating the models


In [ ]:
import torchvision
from torch import nn

OUT_FEATURES = len(resnet18_class_names)

def create_resnet18():
  # 1. Get the base model with pretrained weights and send it to target device
  weights = torchvision.models.ResNet18_Weights.DEFAULT
  model = torchvision.models.resnet18(weights=weights).to(device)

  # 2. Freeze the base model layers
  for param in model.parameters():
    param.requires_grad = False

  # 3. Set the seeds
  set_seeds()

  # 4. Change the classifier head
  model.fc = nn.Linear(in_features=512, out_features=37, bias=True)


  # 5. give the model a name
  model.name = "resnet18"
  print(f"[INFO] Created new {model.name} model.")
  return model
def create_resnet34():
    weights = torchvision.models.ResNet34_Weights.DEFAULT
    model = torchvision.models.resnet34(weights=weights).to(device)

    # Freeze all layers
    for param in model.parameters():
        param.requires_grad = False

    set_seeds()

    model.fc = nn.Linear(in_features=512, out_features=OUT_FEATURES, bias=True)

    return model

# Create an EffNetB0 feature extractor
def create_effnetb0():
  # 1. Get the base model with pretrained weights and sent it to target device
  weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
  model = torchvision.models.efficientnet_b0(weights=weights).to(device)

  # 2. Freeze the base model Layers
  for param in model.features.parameters():
    param.requires_grad = False

  # 3. Set the seeds
  set_seeds()

  # 4. Change the classfier head
  model.classifier = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features=1280, out_features=OUT_FEATURES, bias=True))

  # 5. Give the model a name
  model.name = "effnetb0"
  print(f"[INFO] Created new {model.name} model.")
  return model

# Create an EffNetB2 feature extractor
def create_effnetb2():
  # 1. Get the base model with pretrained weights and send to target device ]
  weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  model = torchvision.models.efficientnet_b2(weights=weights).to(device)

  # 2. Freeze the base model layers
  for param in model.features.parameters():
    param.requires_grad = False

  # 3. Set the seeds
  set_seeds()

  # 4. Change the classifier head
  model.classifier = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features=1408, out_features=OUT_FEATURES, bias=True))

  # 5. Give the model a name
  model.name = "effnetb2"
  print(f"[INFO] Created new {model.name} model.")
  return model

# Create a MobileNetV2 feature extractor
def create_mobilenetv2():
  # 1. Get the base model with pretrained weights and send to target device
  weights = torchvision.models.MobileNet_V2_Weights.DEFAULT
  model = torchvision.models.mobilenet_v2(weights=weights).to(device)

  # 2. Freeze the base model layers
  for param in model.features.parameters():
    param.requires_grad = False

  # 3. Set the seeds
  set_seeds()

  #4. Change the classifier head
  model.classifier = nn.Sequential(
      nn.Dropout(p=0.2, inplace=True),
      nn.Linear(in_features=1280, out_features=OUT_FEATURES, bias=True))
  # 5. Give the model a name
  model.name = "mobilenetv2"
  print(f"[INFO] Created new {model.name} model.")
  return model


In [ ]:
resnet18 = create_resnet18()
resnet34 = create_resnet34()
effnetb0 = create_effnetb0()
effnetb2 = create_effnetb2()
mobilenetv2 = create_mobilenetv2()

In [ ]:
summary(mobilenetv2, (32, 3, 224, 224))

### 1.5 Create Writer

This will allow us to track the results from our experiment on the tensorboard

In [ ]:
!pip install -q tensorboard

In [ ]:
from torch.utils.tensorboard import SummaryWriter
def create_writer(experiment_name: str,
                  model_name: str,
                  extra: str=None) -> SummaryWriter():

  """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

    log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

    Where timestamp is the current date in YYYY-MM-DD format.

    Args:
        experiment_name (str): Name of experiment.
        model_name (str): Name of model.
        extra (str, optional): Anything extra to add to the directory. Defaults to None.

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

    Example usage:
        # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
        writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb2",
                               extra="5_epochs")
        # The above is the same as:
        writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
    """

  from datetime import datetime
  import os

  # Get timestamp of current date (all experiments on certain day like in same folder)
  timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

  if extra:
    log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
  else:
    log_dir = os.path.join("runs", timestamp, experiment_name, model_name)

  print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
  return SummaryWriter(log_dir=log_dir)

### 1.6 Create training function

In [ ]:
from going_modular.going_modular.engine import train_step, test_step
from typing import Dict, List, Tuple
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter



def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          writer: SummaryWriter,
          device: torch.device) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]}
    For example if training for epochs=2:
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

      # Add loss results to SummaryWriter
        writer.add_scalars(main_tag="Loss",
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch)

        # Track the PyTorch model architecture
        writer.add_graph(model=model,
                         # Pass in an example input
                         input_to_model  = torch.randn(32, 3, 224, 224).to(device))
        # Close the writer
        writer.close()

    # Return the filled results at the end of the epochs
    return results


### 1.7 Choose loss function and Optimizer

## 2. Training and Tracking models

In [ ]:
# 1. Create epochs list
epochs = [15,25]


# Create dictionary of our DataLoaders
dataloaders = {
    "resnet18_dataloader": resnet18_train_dataloader
}

model_dataloader_map = {
    "resnet18": "resnet18_dataloader"
}


In [ ]:
from going_modular.going_modular.utils import save_model

# 1. Set the random seeds
set_seeds(seed=42)

# 2. Keep track of experiment numbers
experiment_number = 0

# 3. Loop through each model and its corresponding DataLoader name
for model_name, dataloader_name in model_dataloader_map.items():
    train_dataloader = dataloaders[dataloader_name]  # Access the DataLoader from its name

    # 4. Loop through each number of epochs
    for epoch in epochs:
        experiment_number += 1
        print(f"[INFO] Experiment number: {experiment_number}")
        print(f"[INFO] Model: {model_name}")
        print(f"[INFO] DataLoader: {dataloader_name}")
        print(f"[INFO] Number of epochs: {epoch}")

        # 5. Select and create the model (make sure this is model *constructor*, not pretrained object)
        if model_name == "resnet18":
            selected_model = create_resnet18()
        elif model_name == "resnet34":
            selected_model = create_resnet34()
        elif model_name == "effnetb0":
            selected_model = create_effnetb0()
        elif model_name == "effnetb2":
            selected_model = create_effnetb2()
        elif model_name == "mobilenetv2":
            selected_model = create_mobilenetv2()
        else:
            raise ValueError(f"[ERROR] Model '{model_name}' is not supported.")

        # 6. Create a new loss and optimizer for every model
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(selected_model.parameters(), lr=1e-3)

        # 7. Train target model with target dataloaders and track experiments
        train(model=selected_model,
              train_dataloader=train_dataloader,
              test_dataloader=mobile_test_dataloader,
              optimizer=optimizer,
              loss_fn=loss_fn,
              epochs=epoch,
              device=device,
              writer=create_writer(
                  experiment_name=dataloader_name,
                  model_name=model_name,
                  extra=f"{epoch}_epochs"
              ))

        # 8. Save the model to file
        save_filepath = f"Resistor_Predictor_{model_name}_{dataloader_name}_{epoch}_epochs.pth"
        save_model(model=selected_model,
                   target_dir="models",
                   model_name=save_filepath)
        print("-" * 50 + "\n")


It seems that the best accuracy my models can produce is an accuracy of up to 70%, The classifier layer may have been trained as much as possible.

 Let's try a phased approach for training the model. We will unfreeze the base model layers and train them after training the classifier layer


## Two-Phase Training Loop

Phase 1:
 * Train **only the classifier head** (with frozen base)

Phase 2:
  * Unfreeze the base, then fine-tune the **entire model at a lower learning rate**

In [ ]:
# Reinstantiate our models
resnet18 = create_resnet18()
resnet34 = create_resnet34()
effnetb0 = create_effnetb0()
effnetb2 = create_effnetb2()
mobilenetv2 = create_mobilenetv2()

In [ ]:
from pathlib import Path
import torch

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str,
               test_loss: float = None,
               test_accuracy: float = None):
    """
    Saves a PyTorch model and optional metrics to a target directory.

    Args:
        model: The trained PyTorch model.
        target_dir: Directory to save the model file in.
        model_name: Filename for the model (should end with .pth or .pt).
        test_loss: Optional test loss to save with the model.
        test_accuracy: Optional test accuracy to save with the model.
    """
    # Create target directory if it doesn't exist
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), \
        "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Prepare checkpoint dictionary
    checkpoint = {
        "model_state_dict": model.state_dict(),
    }

    if test_loss is not None:
        checkpoint["test_loss"] = test_loss
    if test_accuracy is not None:
        checkpoint["test_accuracy"] = test_accuracy

    # Save checkpoint
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(checkpoint, model_save_path)


In [ ]:

# 1. Set the random seeds
set_seeds(seed=42)

# 2. Keep track of experiment numbers
experiment_number = 0

# 3. Loop through each Dataloader
for model_name, dataloader_name in model_dataloader_map.items():
    train_dataloader = dataloaders[dataloader_name]  # Access the DataLoader from its name
    # 4. Loop through each number of epochs
    for epoch in epochs:
        experiment_number += 1
        print(f"[INFO] Experiment number: {experiment_number}")
        print(f"[INFO] Model: {model_name}")
        print(f"[INFO] DataLoader: {dataloader_name}")
        print(f"[INFO] Number of epochs per phase: {epoch}")

        # 5. Select and create the model (frozen base)
        if model_name == "resnet18":
            selected_model = create_resnet18()
        elif model_name == "resnet34":
            selected_model = create_resnet34()
        elif model_name == "effnetb0":
            selected_model = create_effnetb0()
        elif model_name == "effnetb2":
            selected_model = create_effnetb2()
        elif model_name == "mobilenetv2":
            selected_model = create_mobilenetv2()
        else:
            raise ValueError(f"[ERROR] Model '{model_name}' is not supported.")

        # ----- Phase 1: Train classifier head only -----
        print("[PHASE 1] Training classifier head (frozen base)...")
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(selected_model.parameters(), lr=1e-3)

        train(model=selected_model,
              train_dataloader=train_dataloader,
              test_dataloader=mobile_test_dataloader,
              optimizer=optimizer,
              loss_fn=loss_fn,
              epochs=epoch,
              device=device,
              writer=create_writer(
                  experiment_name=f"{dataloader_name}_phase1",
                  model_name=model_name,
                  extra=f"{epoch}_epochs"
              ))

         # ----- Phase 2: Fine-tune full model -----
        print("[PHASE 2] Fine-tuning entire model (unfreezing base)...")
        for param in selected_model.parameters():
            param.requires_grad = True  # unfreeze all layers

        # Use a smaller learning rate for fine-tuning
        optimizer_finetune = torch.optim.Adam(selected_model.parameters(), lr=1e-5)

        train(model=selected_model,
              train_dataloader=train_dataloader,
              test_dataloader=mobile_test_dataloader,
              optimizer=optimizer_finetune,
              loss_fn=loss_fn,
              epochs=epoch,
              device=device,
              writer=create_writer(
                  experiment_name=f"{dataloader_name}_phase2",
                  model_name=model_name,
                  extra=f"{epoch}_epochs_finetune"
              ))

        # 6. Save the model to file
        import os
        os.makedirs("/content/drive/MyDrive/resistor_predictor_models", exist_ok=True)


        save_filepath = f"Resistor_Predictor_{model_name}_{dataloader_name}_{epoch*2}_epochs.pth"
        save_model(model=selected_model,
                   target_dir="/content/drive/MyDrive/resistor_predictor_models",
                   model_name=save_filepath)
        print("-" * 50 + "\n")


In [ ]:
%load_ext tensorboard
%tensorboard --logdir drive/MyDrive/resistor_predictor_models

## 3. Load Saved Models

In [ ]:
def inspect_and_load(model, filepath):
    checkpoint = torch.load(filepath, map_location=device)
    if isinstance(checkpoint, dict) and "model_state_dict" in checkpoint:
        print(f"[INFO] '{filepath}' is a checkpoint with 'model_state_dict'")
        model.load_state_dict(checkpoint["model_state_dict"])
    elif isinstance(checkpoint, dict):
        print(f"[INFO] '{filepath}' is a plain state_dict (dictionary of weights)")
        model.load_state_dict(checkpoint)
    else:
        raise ValueError(f"[ERROR] Unknown checkpoint format in {filepath}")
    return model


In [ ]:
resnet18_loaded = inspect_and_load(create_resnet18(), "/content/drive/MyDrive/resistor_backup/resistor_predictor_models/Resistor_Predictor_resnet18_resnet18_dataloader_50_epochs.pth")
resnet34_loaded = inspect_and_load(create_resnet34(), "/content/drive/MyDrive/resistor_backup/resistor_predictor_models/Resistor_Predictor_resnet34_resnet34_dataloaders_50_epochs.pth")
mobilenetv2_loaded = inspect_and_load(create_mobilenetv2(), "/content/drive/MyDrive/resistor_backup/resistor_predictor_models/Resistor_Predictor_mobilenetv2_mobilenetv2_dataloaders_50_epochs.pth")
effnetb0_loaded = inspect_and_load(create_effnetb0(), "/content/drive/MyDrive/resistor_backup/resistor_predictor_models/Resistor_Predictor_effnetb0_effnetb0_dataloaders_50_epochs.pth")
effnetb2_loaded = inspect_and_load(create_effnetb2(), "/content/drive/MyDrive/resistor_backup/resistor_predictor_models/Resistor_Predictor_effnetb2_effnetb2_dataloaders_50_epochs.pth")


## 4. Make Predictions with our models

In [ ]:
loaded_models = {"resnet18":resnet18_loaded,
                 "resnet34": resnet34_loaded,
                 "mobilenetv2":mobilenetv2_loaded,
                 "effnetb0":effnetb0_loaded,
                 "effnetb2":effnetb2_loaded}

loaded_model_transforms = {"resnet18": resnet18_transforms,
"resnet34":resnet34_transforms,
"mobilenetv2":effnetb0_transforms,
"effnetb0":effnetb2_transforms,
"effnetb2":mobilenetv2_transforms}


In [ ]:
import pathlib
import torch

from PIL import Image
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing  import List, Dict

# 1. Create a function to return a list of dictionaries with sample, truth label, prediction, prediction probability and prediction time
def pred_and_store(paths: List[pathlib.Path],
                   model: torch.nn.Module,
                   transform: torchvision.transforms,
                   class_names: List[str],
                   device: str = "cuda" if torch.cuda.is_available() else "cpu") -> List[Dict]:

    # 2. Create an empty list to store prediction dictionaries
    pred_list = []

    # 3. Loop through target paths
    for path in tqdm(paths):

      # 4. Create empty dictionary to store prediction information for each sample
      pred_dict = {}

      # 5. Get the sample path and ground truth class name
      pred_dict["image_path"] = path
      class_name = path.parent.stem
      pred_dict["class_name"] = class_name


      # 6. Start the prediction timer
      start_time = timer()

      # 7. Open image path
      img = Image.open(path)

      # 8. Transform the image, add batch dimension and put image on target device
      transformed_image = transform(img).unsqueeze(0).to(device)

      # 9. Prepare model for inference by sending it to target device and turning one eval() mode
      model.to(device)
      model.eval()

      # 10. Get prediction probality, prediction label and prediction class
      with torch.inference_mode():
        pred_logit = model(transformed_image) # perform inference on target sample
        pred_prob = torch.softmax(pred_logit, dim=1) # turn logits into prediction probailities
        pred_label = torch.argmax(pred_prob, dim=1)  # turn prediction probailities into prediction label
        pred_class = class_names[pred_label.cpu()]   # hardcode prediction class to be on CPU


        #11. Make sure things in the dictionary are on CPU (required for inspecting predictions later on)
        with torch.inference_mode():
          pred_dict["pred_prob"] = round(pred_prob.unsqueeze(0).max().cpu().item(), 4)
          pred_dict["pred_class"] = pred_class

          # 12. End the timer and calculate the time per prediction
          end_time = timer()
          pred_dict["time_for_pred"] = end_time-start_time

        # 13. Does the pred match the true label?
        pred_dict["correct"] = class_name == pred_class

        # 14. Add the dictionary to the list of preds
        pred_list.append(pred_dict)

  # 15. Return list of prediction dictionares
    return pred_list

In [ ]:
from pathlib import Path


test_data_paths = list(Path(test_dir).glob("*/*.jpg") )

all_cpu_results = {}

for loaded_model_name, loaded_model in tqdm(loaded_models.items()):
  cpu_test_results = pred_and_store(paths=test_data_paths,
                    model=loaded_model,
                    transform=loaded_model_transforms[loaded_model_name],
                    class_names=mobile_class_names,
                    device="cpu") # Make predictions on the cpu to replicate a mobile device

  # Add the reults into final dictionary
  all_cpu_results[loaded_model_name] = cpu_test_results


In [ ]:
# Turn the all_cpu_results into a DataFrame
import pandas as pd

all_cpu_results_df = pd.DataFrame(all_cpu_results)
all_cpu_results_df.head()

In [ ]:
# Flatten results into a list of dictionaries, adding model name to each entry
flat_results = []

for model_name, preds in all_cpu_results_df.items():
    for pred in preds:
        pred["model_name"] = model_name
        flat_results.append(pred)

# Convert to DataFrame
flat_df = pd.DataFrame(flat_results)

In [ ]:
# Determine the predictions accuracy results and average prediction time
model_prediction_stats =flat_df.groupby("model_name")["correct"].mean(),flat_df.groupby("model_name")["time_for_pred"].mean()
model_prediction_stats = pd.DataFrame(model_prediction_stats).transpose()
model_prediction_stats["correct"] = round(model_prediction_stats["correct"]*100,2)
model_prediction_stats["time_for_pred"] = round(model_prediction_stats["time_for_pred"],4)
model_prediction_stats.columns = ["Accuracy(%)", "Average Prediction Time(s)"]


In [ ]:
# Find the size of each of the models



resnet18_path="/content/drive/MyDrive/resistor_backup/resistor_predictor_models/Resistor_Predictor_resnet18_resnet18_dataloaders_50_epochs.pth"
resnet34_path="/content/drive/MyDrive/resistor_backup/resistor_predictor_models/Resistor_Predictor_resnet34_resnet34_dataloaders_50_epochs.pth"
mobilenetv2_path = "/content/drive/MyDrive/resistor_backup/resistor_predictor_models/Resistor_Predictor_mobilenetv2_mobilenetv2_dataloaders_50_epochs.pth"
effnetb0_path ="/content/drive/MyDrive/resistor_backup/resistor_predictor_models/Resistor_Predictor_effnetb0_effnetb0_dataloaders_50_epochs.pth"
effnetb2_path = "/content/drive/MyDrive/resistor_backup/resistor_predictor_models/Resistor_Predictor_effnetb2_effnetb2_dataloaders_50_epochs.pth"

# Model path dictionary
loaded_model_paths = {"resnet18": resnet18_path,
"resnet34":resnet34_path,
"mobilenetv2":effnetb0_path,
"effnetb0":effnetb2_path,
"effnetb2":mobilenetv2_path}

# Model size dictironary
loaded_model_size = {}

for model_name, model_path in loaded_model_paths.items():
  model_size = os.path.getsize(model_path)//(1024*1024)
  loaded_model_size[model_name] = model_size

model_prediction_stats = model_prediction_stats.join(pd.DataFrame(loaded_model_size, index=["Model Size(MB)"]).transpose())
model_prediction_stats



In [ ]:
model_prediction_stats.drop(["resnet18", "resnet34","mobilenetv2", "effnetb0", "effnetb2"], axis=1, inplace=True)

In [ ]:
model_prediction_stats

Model Selection for Resistor Value Predictor (Visual CNN)

| Model Name   | Accuracy (%) | Prediction Time (s) | Model Size (MB) |
|--------------|--------------|--------------------|-----------------|
| effnetb0     | 83.90        | 0.0435              | 30              |
| effnetb2     | 80.68        | 0.0478              | 8               |
| mobilenetv2  | 81.69        | 0.0318              | 15              |
| resnet18     | 92.03        | 0.0283              | 42              |
| resnet34     | 94.58        | 0.0418              | 81              |

### Key Considerations:
1. **Accuracy** — Critical for correct resistor value predictions.
2. **Prediction Time** — Affects real-time user experience.
3. **Model Size** — Impacts deployment cost, loading times, and mobile-friendliness.

### Analysis:
- **ResNet34**:
  - **Highest accuracy (94.58%)**
  - **Largest size (81MB)**
  - Moderate prediction time.
- **ResNet18**:
  - **High accuracy (92.03%)**
  - **Fastest prediction time (0.0283s)**
  - Medium size (42MB)
- **MobileNetV2**:
  - Lightweight (15MB)
  - Fast inference
  - Lower accuracy (81.69%)
- **EffNetB0 / EffNetB2**:
  - Moderate size models
  - Accuracy below 84%

### Recommendation:
- **Best Choice → ResNet18**
  - High accuracy (92.03%)
  - Fastest inference (0.0283s)
  - Reasonable size (42MB)
  
- **If Model Size is Critical (e.g., mobile-first app)**:
  - Consider **MobileNetV2** (15MB), but expect a ~10% drop in accuracy.

---

### Deployment Suggestion:
- **For Web/Cloud Deployment**: ResNet18 is optimal.
- **For Edge Devices (strict size constraints)**: MobileNetV2 is acceptable.

---




## 5. Deploy the ResNet18 Model into Gradio

In [ ]:
# Import/Install Gradio

try:
  import gradio as gr
except:
  !pip install gradio
  import gradio as gr

print(f"Gradio version: {gr.__version__}")

In [ ]:
# Put resnet18 on cpu
resnet18_loaded.to("cpu")

# Check the device
next(iter(resnet18_loaded.parameters())).device

### 5.1 Creating a Prediction Function

In [ ]:
from typing import Tuple, Dict
from timeit import default_timer as timer

def predict(img) -> Tuple[Dict, float]:
  """Transforms and performs a prediction on img and returns prediction and time taken
  """

  # start the timer
  start_time = timer()

  # Tranform the target image and add a batch dimension
  img = resnet18_transforms(img).unsqueeze(0)

  # Put model into evaluation mode and turn on inference mode
  resnet18_loaded.eval()
  with torch.inference_mode():
    # Pass the tranformed image through the model and turn the prediction logits into prediction probabilities
    pred_probs = torch.softmax(resnet18_loaded(img), dim=1)

  # Create a prediction label and prediction probaility dictionary for each prediction dictionary for each prediction class (this is the required format got Gradio's output parameter)
  pred_labels_and_probs = {resnet18_class_names[i]: float(pred_probs[0][i]) for i in range(len(resnet18_class_names))}

  # Calculate the prediction time
  pred_time = round(timer() - start_time, 5)

  # Return the prediction dictionary and prediction time
  return pred_labels_and_probs, pred_time


In [ ]:
import random
from PIL import Image

# Get a list of all test image filepaths
test_data_paths = list(Path(test_dir).glob("*/*.jpg") )

# randomly select a test image path
random_image_path = random.sample(test_data_paths, k=1)[0]

# Open the target image
image = Image.open(random_image_path)

# Predict on the target image
pred_dict, pred_time = predict(img=image)
print(f"Prediction label and probaility dictionary: \n{pred_dict}")
print(f"Prediction time: {pred_time} seconds")

### 5.2 Create a list of examples

In [ ]:
# Create a list of example inputs to our Gradio demo
example_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=5)]
example_list

### 5.3 Building Our Gradio Interface

Let's create a Gradio interface to replicate the workflow:

```
input: image -> transform -> predict with EffNetB2 -> output: pred, pred prob, time taken
```

We can do with the [`gradio.Interface()`](https://gradio.app/docs/#interface) class with the following parameters:
* `fn` - a Python function to map `inputs` to `outputs`, in our case, we'll use our `predict()` function.
* `inputs` - the input to our interface, such as an image using [`gradio.Image()`](https://gradio.app/docs/#image) or `"image"`.
* `outputs` - the output of our interface once the `inputs` have gone through the `fn`, such as a label using [`gradio.Label()`](https://gradio.app/docs/#label) (for our model's predicted labels) or number using [`gradio.Number()`](https://gradio.app/docs/#number) (for our model's prediction time).
    * **Note:** Gradio comes with many in-built `inputs` and `outputs` options known as ["Components"](https://gradio.app/docs/#components).
* `examples` - a list of examples to showcase for the demo.
* `title` - a string title of the demo.
* `description` - a string description of the demo.
* `article` - a reference note at the bottom of the demo.

Once we've created our demo instance of `gr.Interface()`, we can bring it to life using [`gradio.Interface().launch()`](https://gradio.app/docs/#launch-header) or `demo.launch()` command.

Easy!


In [ ]:
import gradio as gr

# Create title, desciption and article strings
title = "Resistor Predictor CNN 🎚️"
description = "A ResNet18 feature extractor computer vision model to classify images of varying resistors to their corresponding resistor values"
article = "Created by Murede A"

# Create the Gradio demo
demo =gr.Interface(fn=predict, # mapping function from input to output
                    inputs=gr.Image(type="pil"), # what are the inputs?
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"), # what are the outputs?
                             gr.Number(label="Prediction time (s)")], # our fn has two outputs, therefore we have two outputs
                    examples=example_list,
                    title=title,
                    description=description,
                    article=article)
# Launch the demo!
demo.launch(debug=False, # print errors locally?
            share=True) # generate a publically shareable URL?

## 6. Turning Resistor Predictor CNN into a deployable app

### 6.1 Creating a `demos` folder to store our Resistor Predictor CNN into a deployable app


In [ ]:
import shutil
from pathlib import Path

# Create Resistor Predictor CNN demo path
resistor_predictor_demo_path = Path("demos/resistor_predictor")

# Remove files that might already exist there and create new directory
if resistor_predictor_demo_path.exists():
  print(f"{resistor_predictor_demo_path} directory exists. Removing...")
  shutil.rmtree(resistor_predictor_demo_path)

# If the file doesn't exist, create it anyway
resistor_predictor_demo_path.mkdir(parents=True, exist_ok=True)

# Check what's in the folder
!ls demos/resistor_predictor


### 6.2  Create a folder of example images to use with our Resistor Predictor demo



In [ ]:
import shutil
from pathlib import Path

# 1. Acreate an examples directory
resistor_predictor_example_path = resistor_predictor_demo_path / "examples"
resistor_predictor_example_path.mkdir(parents=True, exist_ok=True)

# 2. Collect three random test dataset image paths
resistor_examples = [
    Path('resistor_dataset/68K_1W/68K_1W_(12).jpg'),
    Path('resistor_dataset/620R_1-4W/620R_1-4W_(3).jpg'),
    Path('resistor_dataset/10R_2W/10R_2W_(5).jpg'),
    Path('resistor_dataset/100R_1-4W/100R_1-4W_(2).jpg'),
    Path('resistor_dataset/150R_1-8W/150R_1-8W_(7).jpg')
]


# 3. Copy the three random images to the examples directory
for example in resistor_examples:
  destination = resistor_predictor_example_path / example.name
  print(f"[INFO] Copying{example} to {destination}")
  shutil.copyfile(src=example, dst=destination)

In [ ]:
import os

# Get example filepaths in a list of lists
example_list = [["examples/" +example] for example in os.listdir(resistor_predictor_example_path)]
example_list

### 6.3 Moving our trainned ResNet18 model to our  Resistor Predictor demo directory

In [ ]:
from pathlib import Path

file_path = Path("/content/drive/MyDrive/resistor_backup/resistor_predictor_models/Resistor_Predictor_resnet18_resnet18_dataloaders_50_epochs.pth")
print(file_path.exists())


In [ ]:
import shutil
from pathlib import Path

# Define the source model path
resnet18_resistor_predictor_model_path = "/content/drive/MyDrive/resistor_backup/resistor_predictor_models/Resistor_Predictor_resnet18_resnet18_dataloader_50_epochs.pth"

# Define temporary copy location (inside /content for example)
temp_copy_path = Path("/content") / Path(resnet18_resistor_predictor_model_path).name

# Define destination path
resistor_predictor_demo_path = Path("demos/resistor_predictor")
resistor_predictor_demo_path.mkdir(parents=True, exist_ok=True)  # make sure it exists
resnet18_resistor_predictor_model_destination = resistor_predictor_demo_path / temp_copy_path.name

# Step 1: Copy the file
try:
    print(f"[INFO] Copying file to temporary location: {temp_copy_path}")
    shutil.copy2(src=resnet18_resistor_predictor_model_path, dst=temp_copy_path)
    print(f"[SUCCESS] File copied to: {temp_copy_path}")
except FileNotFoundError:
    print(f"[ERROR] Source file not found: {resnet18_resistor_predictor_model_path}")
except shutil.Error as e:
    print(f"[ERROR] shutil copy error: {e}")
except Exception as e:
    print(f"[ERROR] Unexpected error during copy: {e}")

# Step 2: Move the copied file to demo directory
try:
    print(f"[INFO] Moving copied file to: {resnet18_resistor_predictor_model_destination}")
    shutil.move(src=temp_copy_path, dst=resnet18_resistor_predictor_model_destination)
    print(f"[SUCCESS] File moved to: {resnet18_resistor_predictor_model_destination}")
except FileNotFoundError:
    print(f"[ERROR] Copied file not found: {temp_copy_path}")
except shutil.Error as e:
    print(f"[ERROR] shutil move error: {e}")
except Exception as e:
    print(f"[ERROR] Unexpected error during move: {e}")


### 6.4 Turning ResNet18 model into a Python script(model.py)



In [ ]:
%%writefile demos/resistor_predictor/model.py

import torch
import torchvision

from torch import nn

def create_resnet18():
  # 1. Get the base model with pretrained weights and send it to target device
  weights = torchvision.models.ResNet18_Weights.DEFAULT
  transforms = weights.transforms()
  model = torchvision.models.resnet18(weights=weights).to("cpu")

  # 2. Freeze the base model layers
  for param in model.parameters():
    param.requires_grad = False

  # 3. Set the seeds
  torch.manual_seed(42)


  # 4. Change the classifier head
  model.fc = nn.Linear(in_features=512, out_features=37, bias=True)

  return model, transforms

### 6.5 Turning Resistor Predictor Gradio app into a Python Script (app.py)


In [ ]:
%%writefile demos/resistor_predictor/app.py

### 1.  Imports and class names setup ###
import gradio as gr
import os
import torch
from pathlib import Path

from model import create_resnet18
import examples
examples = "examples"
from timeit import default_timer as timer
from typing import Tuple, Dict

# Setup class names
class_names = ['100R_1-4W',
 '10R_1W',
 '10R_2W',
 '10_1-4W',
 '11M_1-2W',
 '150R_1-4W',
 '150R_1-8W',
 '15R_1-4W',
 '180K_1-2W',
 '1K_1-4W',
 '1K_2W',
 '1M_1-4W',
 '20K_1-4W',
 '220K_1-4W',
 '220R_2W',
 '22R_1-4W',
 '24K_1-2W',
 '270K_1-4W',
 '27R_1W',
 '2K2_1-4W',
 '2R_1W',
 '330R_1-4W',
 '33K_2W',
 '3R9K_1-4W',
 '4700Mohm',
 '470R_1-4W',
 '470R_1W',
 '4K7_1-4W',
 '56K_1W',
 '5K1_1-4W',
 '5K61-4W',
 '620R_1-4W',
 '68K_1W',
 '6R8_1-4W',
 '7K5_1-4W',
 '820R_1-4W',
 '8K2_1-4W']

 ## 2. Model and transforms preparation ###

# Create ResNet18 model
resnet18, resnet18_transforms = create_resnet18()
resnet18 = resnet18.to("cpu")

# Load saved weights
model_path = Path("Resistor_Predictor_resnet18_resnet18_dataloader_50_epochs.pth")
checkpoint = torch.load(model_path, map_location="cpu")
resnet18.load_state_dict(checkpoint["model_state_dict"])


### 3. Predict function ###

# Create predict function
def predict(img) -> Tuple[Dict, float]:
  """Transforms and performs a prediction on img and returns prediction and time taken
  """

  # start the timer
  start_time = timer()

  # Tranform the target image and add a batch dimension
  img = resnet18_transforms(img).unsqueeze(0)

  # Put model into evaluation mode and turn on inference mode
  resnet18.eval()
  with torch.inference_mode():
    # Pass the tranformed image through the model and turn the prediction logits into prediction probabilities
    pred_probs = torch.softmax(resnet18(img), dim=1)

  # Create a prediction label and prediction probaility dictionary for each prediction dictionary for each prediction class (this is the required format got Gradio's output parameter)
  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

  # Calculate the prediction time
  pred_time = round(timer() - start_time, 5)

  # Return the prediction dictionary and prediction time
  return pred_labels_and_probs, pred_time

### 4. Gradio app ###

# Create title, desciption and article strings
title = "Resistor Predictor CNN 🎚️"
description = "A ResNet18 feature extractor computer vision model to classify images of varying resistors to their corresponding resistor values"
article = "Created by Murede A"

# Gradio interface setup
title = "Resistor Predictor CNN 🎚️"
description = "A ResNet18-based model that classifies resistor images."
article = "Created by Murede A"
examples = "examples"  # Or list of images like [["examples/1.jpg"], ...]

demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Label(num_top_classes=3, label="Predictions"),
             gr.Number(label="Prediction time (s)")],
    examples=examples,
    title=title,
    description=description,
    article=article
)

if __name__ == "__main__":
    demo.launch(debug=False, share=True)

### 6.6 Creating a requirements file for Resistor Predictor

In [ ]:
%%writefile demos/resistor_predictor/requirements.txt
torch==2.6.0
torchvision==0.21.0
gradio==5.31.0


### 7. Uploading our Resistor Predictor app into HuggingFace Spaces

These are all files that we've created!

To begin uploading our files to Hugging Face, let's now download them from Google Colab (or wherever you're running this notebook).

To do so, we'll first compress the files into a single zip folder via the command:

```
zip -r ../resistor_predictor.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"
```

Where:
* `zip` stands for "zip" as in "please zip together the files in the following directory".
* `-r` stands for "recursive" as in, "go through all of the files in the target directory".
* `../resistor_predictor.zip` is the target directory we'd like our files to be zipped to.
* `*` stands for "all the files in the current directory".
* `-x` stands for "exclude these files".

We can download our zip file from Google Colab using [`google.colab.files.download("demos/resistor_predictor.zip")`](https://colab.research.google.com/notebooks/io.ipynb) (we'll put this inside a `try` and `except` block just in case we're not running the code inside Google Colab, and if so we'll print a message saying to manually download the files).

Let's try it out!

In [ ]:
# Change into and then zip the resistor_predictor folder but exclude certain files
!cd demos/resistor_predictor && zip -r ../resistor_predictor.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

# Download the zipped Resistor  Predictor app (if running in Google Colab)
try:
  from google.colab import files
  files.download("demos/resistor_predictor.zip")
except:
  print("Not running in Google Colab, can't use google.colab.files.download(), please manually download.")

### 7.2 Running Resistor Predictor demo locally

If you download the `resistor_predictor.zip` file, you can test it locally by:
1. Unzipping the file.
2. Opening terminal or a command line prompt.
3. Changing into the `resistor_predictor` directory (`cd resistor_predictor`).
4. Creating an environment (`python -m venv env`).
5. Activating the environment (`env\Scripts\activate`).
5. Installing the requirements (`pip install -r requirements.txt`, the "`-r`" is for recursive).
    * **Note:** This step may take 5-10 minutes depending on your internet connection. And if you're facing errors, you may need to upgrade `pip` first: `pip install --upgrade pip`.
6. Run the app (`python app.py`).

This should result in a Gradio demo just like the one we built above running locally on your machine at a URL such as `http://127.0.0.1:7860/`.

> **Note:** If you run the app locally and you notice a `flagged/` directory appear, it contains samples that have been "flagged".
>
> For example, if someone tries the demo and the model produces an incorrect result, the sample can be "flagged" and reviewed for later.
>
> For more on flagging in Gradio, see the [flagging documentation](https://gradio.app/docs/#flagging).

### 9.3 Uploading to Hugging Face

We've verified our Resistor Predictor  app works locally, however, the fun of creating a machine learning demo is to show it to other people and allow them to use it.

To do so, we're going to upload our Resistor PRedictor demo to Hugging Face.

> **Note:** The following series of steps uses a Git (a file tracking system) workflow. For more on how Git works, I'd recommend going through the [Git and GitHub for Beginners tutorial](https://youtu.be/RGOj5yH7evk) on freeCodeCamp.

1. [Sign up](https://huggingface.co/join) for a Hugging Face account.
2. Start a new Hugging Face Space by going to your profile and then [clicking "New Space"](https://huggingface.co/new-space).
    * **Note:** A Space in Hugging Face is also known as a "code repository" (a place to store your code/files) or "repo" for short.
3. Give the Space a name,
4. Select a license (I used [MIT](https://opensource.org/licenses/MIT)).
5. Select Gradio as the Space SDK (software development kit).
   * **Note:** You can use other options such as Streamlit but since our app is built with Gradio, we'll stick with that.
6. Choose whether your Space is it's public or private (I selected public since I'd like my Space to be available to others).
7. Click "Create Space".
8. Clone the repo locally by running something like: `git clone https://huggingface.co/spaces/[YOUR_USERNAME]/[YOUR_SPACE_NAME]` in terminal or command prompt.
    * **Note:** You can also add files via uploading them under the "Files and versions" tab.
9. Copy/move the contents of the downloaded `resistor_predictor` folder to the cloned repo folder.
10. To upload and track larger files (e.g. files over 10MB or in our case, our PyTorch model file) you'll need to [install Git LFS](https://git-lfs.github.com/) (which stands for "git large file storage").
11. After you've installed Git LFS, you can activate it by running `git lfs install`.
12. In the `resistor_predictor` directory, track the files over 10MB with Git LFS with `git lfs track "*.file_extension"`.
    * Track EffNetB2 PyTorch model file with `git lfs track "Resistor_Predictor_resnet18_resnet18_dataloaders_50_epochs.pth"`.
13. Track `.gitattributes` (automatically created when cloning from HuggingFace, this file will help ensure our larger files are tracked with Git LFS). You can see an example `.gitattributes` file on the [FoodVision Mini Hugging Face Space](https://huggingface.co/spaces/mrdbourke/foodvision_mini/blob/main/.gitattributes).
    * `git add .gitattributes`
14. Add the rest of the `resistor_predictor` app files and commit them with:
    * `git add *`
    * `git commit -m "first commit"`
15. Push (upload) the files to Hugging Face:
    * `git push`
16. Wait 3-5 minutes for the build to happen (future builds are faster) and your app to become live!

If everything worked, you should see a live running example of our FoodVision Mini Gradio demo like the one here: https://huggingface.co/spaces/mrdbourke/foodvision_mini

And we can even embed our FoodVision Mini Gradio demo into our notebook as an [iframe](https://gradio.app/sharing_your_app/#embedding-with-iframes) with [`IPython.display.IFrame`](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html#IPython.display.IFrame) and a link to our space in the format `https://hf.space/embed/[YOUR_USERNAME]/[YOUR_SPACE_NAME]/+`.

In [ ]:
# IPython is a library to help make Python interactive
from IPython.display import IFrame

# Embed Resistor Predictor Gradio demo
